In [1]:
import os
import sys
import xarray as xr
import numpy as np
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from pathlib import Path
import cartopy.crs as ccrs
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
mcs_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/mcs_tracks_non2mcs')

In [4]:
year_list = np.arange(2001,2011)

for year in year_list:
    print(year)
    # merge all tracks into a single xarray.dataset file 
    ds_MCS = xr.open_dataset(mcs_dir / 'mcs_tracks_non2mcs_{}.tropics30NS.full.nc'.format(year))
    
    ds_MCSphase = []
    track_list = []
    for i in range(len(ds_MCS.tracks)):
        tmp = ds_MCS.isel(tracks=i)
        idt_phase = [0,
                     tmp.idt_mcs_init.values,
                     tmp.idt_mcs_grow.values,
                     tmp.idt_mcs_mature.values,
                     tmp.idt_mcs_decay.values,
                     tmp.idt_mcs_end.values]    
        tmp_phase = tmp.isel(times = idt_phase)
        tmp_phase['times'] = np.arange(6)
        tmp_phase = tmp_phase.rename({'times':'time'})
        
        track_list.append(int(tmp.tracks.values))
        ds_MCSphase.append(tmp_phase[['base_time','meanlat','meanlon']])  
    ds_MCSphase_xr = xr.concat(ds_MCSphase, pd.Index(track_list, name='tracks'))
    
    meanlon = ds_MCSphase_xr['meanlon']
    meanlon_re = meanlon.where(meanlon >= 0, meanlon + 360)
    ds_MCSphase_xr['meanlon'] = meanlon_re
    
    out_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/mcs_tracks_non2mcs/featenv_track_input')
    ds_MCSphase_xr.to_netcdf(out_dir / 'MCS_FLEXTRKR_tropics30NS.{}.nc'.format(year)
                         , encoding={'meanlon': {'dtype': 'float32'}, 'meanlat': {'dtype': 'float32'}})

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010


In [ ]:
track_sel = ds_MCSphase_xr.tracks.values
np.random.shuffle(track_sel) # shuffle to avoid I/O traffics due to competition

In [ ]:
ds_MCSphase_xr.tracks